In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 22253786
paper_name = 'blackman_nislow_2012' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/pone.0029798.s003.xlsx', sheet_name = 'elescomol')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 6268 x 19


In [7]:
original_data.head()

,strain,Gene,log2,MITO,NO,mitoL,resp,metal,trna,trans,MGM,transp,splice,stress,het,Description,GO_function,GO_process,GO_component
0,YGL165C::chr7_2,CUP2,4.241930,y,NaN,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Partially overlaps the verified ORF CUP2/YGL166W,DNA binding (IEA); copper ion binding (IEA); l...,autophagy (RCA); regulation of transcription (...,"nucleus (IEA, IDA)"
1,YOR158W::chr15_2,PET123,3.235505,y,NaN,NaN,NaN,NaN,NaN,y,NaN,NaN,NaN,NaN,NaN,Mitochondrial ribosomal protein of the small s...,structural constituent of ribosome (IPI),ER-nucleus signaling pathway (RCA); mitochondr...,mitochondrial small ribosomal subunit (IPI); m...
2,YOL027C::chr15_5,MDM38,3.172985,y,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Mitochondrial inner membrane protein, involved...",molecular_function (ND),cellular potassium ion homeostasis (IMP); mito...,integral to membrane (IEA); membrane (IEA); mi...
3,YML081W::chr13_1b,TDA9,3.126410,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Putative protein of unknown function; green fl...,DNA binding (IEA); metal ion binding (IEA); nu...,biological_process (ND),"intracellular (IEA); nucleus (IEA, IDA)"
4,YER154W::chr00_5,OXA1,3.057050,y,NaN,y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Mitochondrial inner membrane insertase, mediat...",membrane insertase activity (IMP); ribosome bi...,establishment of protein localization (RCA); i...,integral to membrane (IEA); membrane (IEA); mi...


In [8]:
original_data['orf'] = original_data['strain'].apply(lambda x: x.split(':')[0])

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [strain, Gene, log2, MITO, NO, mitoL, resp, metal, trna, trans, MGM, transp, splice, stress, het, Description, GO_function, GO_process, GO_component, orf]
Index: []


In [12]:
original_data['data'] = original_data['log2']

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(6037, 1)

# Separate HOM and HET using today's HOM collection from Open Biosystems

In [19]:
hom = pd.read_excel('extras/Homozygous_diploid_obs_v7.0.xlsx', sheet_name='DATA')

In [21]:
hom.head()

,Record number,ORF,Batch,Collection,Plate,Row,Col,Comment
0,30916,YHL047C,chr8_1,Hom Dip,1,A,2,NaN
1,30917,YHL046C,chr8_1,Hom Dip,1,A,3,NaN
2,30918,YHL045W,chr8_1,Hom Dip,1,A,4,NaN
3,30919,YHL044W,chr8_1,Hom Dip,1,A,5,NaN
4,30920,YHL043W,chr8_1,Hom Dip,1,A,6,NaN


In [23]:
hom['orf'] = hom['ORF'].astype(str)

In [24]:
hom['orf'] = clean_orf(hom['orf'])

In [25]:
hom['orf'] = translate_sc(hom['orf'], to='orf')

In [26]:
# Make sure everything translated ok
t = looks_like_orf(hom['orf'])
print(hom.loc[~t,])

Empty DataFrame
Columns: [Record number, ORF, Batch, Collection, Plate, Row, Col, Comment, orf]
Index: []


In [27]:
hom_orfs = hom['orf'].unique()

In [29]:
hom_orfs_in_data = [orf for orf in hom_orfs if orf in original_data.index.values]

In [31]:
original_data1 = original_data.loc[hom_orfs_in_data,:].copy()
original_data2 = original_data.drop(index=hom_orfs_in_data, inplace=False)

In [32]:
original_data1.shape

(4877, 1)

In [33]:
original_data2.shape

(1160, 1)

In [34]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

# Prepare the final dataset

In [35]:
data = original_data.copy()

In [36]:
dataset_ids = [86,87]
datasets = datasets.reindex(index=dataset_ids)

In [37]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [38]:
data.head()

dataset_id,86,87
data_type,value,value
orf,,
YAL001C,NaN,-0.405303
YAL002W,NaN,NaN
YAL003W,NaN,-0.140374
YAL004W,-0.140326,NaN
YAL005C,-0.062585,NaN


## Subset to the genes currently in SGD

In [39]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 25


In [40]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,86,87
,data_type,value,value
gene_id,orf,,
1,YAL001C,NaN,-0.405303
2,YAL002W,NaN,NaN
3,YAL003W,NaN,-0.140374
1863,YAL004W,-0.140326,NaN
4,YAL005C,-0.062585,NaN


# Normalize

In [41]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [42]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [43]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,86,87,86,87
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
1,YAL001C,NaN,-0.405303,NaN,-0.895733
2,YAL002W,NaN,NaN,NaN,NaN
3,YAL003W,NaN,-0.140374,NaN,-0.209298
1863,YAL004W,-0.140326,NaN,-0.069468,NaN
4,YAL005C,-0.062585,NaN,0.079165,NaN


# Print out

In [44]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [45]:
from IO.save_data_to_db3 import *

In [46]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 22253786...
Inserting the new data...


100%|██████████| 2/2 [00:17<00:00,  8.92s/it]

Updating the data_modified_on field...
